In [ ]:
# ===========================================
# 📊 BOOKS_RATINGS_BY_SEQUENCE (Camada Gold)
# ===========================================
from pyspark.sql.functions import col, avg, count, row_number, round
from pyspark.sql.window import Window

# 1. Carregar livros e avaliações já limpos (silver)
df_books = spark.table("book_silver.books")
df_ratings = spark.table("book_silver.ratings")

# 2. Calcular nota média e contagem de avaliações por livro
ratings_agg = df_ratings.groupBy("isbn").agg(
    avg("rating").alias("avg_rating"),
    count("rating").alias("num_ratings")
)

# 3. Juntar com livros para trazer título, autor, ano
books_with_ratings = df_books.join(ratings_agg, on="isbn", how="inner")

# 4. Criar sequência de publicação por autor (ordem crescente de ano)
window_spec = Window.partitionBy("author").orderBy("year")

books_with_sequence = books_with_ratings.withColumn(
    "book_order", row_number().over(window_spec)
)

# 5. Selecionar e organizar as colunas finais
df_gold = books_with_sequence.select(
    "author", "title", "year", "book_order", "avg_rating", "num_ratings"
)

# 6. Salvar como tabela gold
df_gold.write.mode("overwrite").saveAsTable("book_gold.books_ratings_by_sequence")





# ===========================================
# 📊 GOLD - TENDÊNCIA DE AVALIAÇÃO POR SEQUÊNCIA (1º ao 20º livro)
# ===========================================
from pyspark.sql.functions import avg, count, round, col

# Carregar a tabela de sequência de livros por autor
df_sequence = spark.table("book_gold.books_ratings_by_sequence")

# Filtrar livros até a 20ª posição
df_limit_20 = df_sequence.filter(col("book_order") <= 20)

# Calcular a média das notas por posição de lançamento
df_trend = df_limit_20.groupBy("book_order").agg(
    round(avg("avg_rating"), 3).alias("avg_rating_por_posicao"),
    count("avg_rating").alias("total_livros")
).orderBy("book_order")

# Exibir resultado em tabela
display(df_trend)

# Salvar como tabela gold auxiliar
df_trend.write.mode("overwrite").saveAsTable("book_gold.rating_trend_by_sequence")

# ===========================================
# 📈 VISUALIZAÇÃO
# ===========================================
import matplotlib.pyplot as plt

# Converter para pandas para plotar
df_pd = df_trend.toPandas()

plt.figure(figsize=(10, 5))
plt.plot(df_pd["book_order"], df_pd["avg_rating_por_posicao"], marker='o')
plt.title("Tendência da Avaliação Média por Ordem de Lançamento")
plt.xlabel("Posição do Livro na Carreira do Autor")
plt.ylabel("Nota Média")
plt.grid(True)
plt.tight_layout()
plt.show()

#1. Tendência de nota por ordem de lançamento

#A análise indica que os primeiros livros publicados por um autor tendem a receber avaliações mais #altas, especialmente os dois primeiros. A partir do terceiro ou quarto lançamento, a média de notas #tende a se estabilizar. Isso pode estar relacionado à expectativa inicial dos leitores ou à #dificuldade de manter um alto nível de inovação ao longo da carreira. É importante destacar que #esta análise não considera o tempo entre lançamentos e pode ter viés para autores com poucos livros #publicados.





# ===========================================
# 📊 GOLD - INTERVALO ENTRE LIVROS VS NOTA
# ===========================================
from pyspark.sql.functions import col, avg, round, lag
from pyspark.sql.window import Window

# 1. Carregar tabela gold com sequências
books_seq = spark.table("book_gold.books_ratings_by_sequence")

# 2. Criar coluna com o ano do livro anterior
window = Window.partitionBy("author").orderBy("book_order")
books_with_lag = books_seq.withColumn("prev_year", lag("year").over(window))

# 3. Calcular intervalo entre lançamentos (em anos)
books_with_diff = books_with_lag.withColumn(
    "years_since_last", col("year") - col("prev_year")
).filter(col("book_order") > 1)

# 4. Salvar tabela com gaps e notas
books_with_diff.write.mode("overwrite").saveAsTable("book_gold.rating_by_release_gap")

# ===========================================
# 📈 VISUALIZAÇÃO DA MÉDIA POR INTERVALO
# ===========================================
from pyspark.sql.functions import count
import matplotlib.pyplot as plt

# Agrupar por intervalo e calcular média das notas
df_gap_trend = books_with_diff.groupBy("years_since_last").agg(
    round(avg("avg_rating"), 2).alias("avg_rating"),
    count("avg_rating").alias("num_livros")
).orderBy("years_since_last")

# Mostrar dados em tabela
display(df_gap_trend)

# Converter para pandas para plotar
df_pd = df_gap_trend.toPandas()

# ===========================================
# 📈 VISUALIZAÇÃO
# ===========================================
plt.figure(figsize=(10, 5))
plt.plot(df_pd["years_since_last"], df_pd["avg_rating"], marker='o')
plt.title("Nota Média por Intervalo de Lançamento entre Livros")
plt.xlabel("Anos desde o último livro")
plt.ylabel("Nota média")
plt.grid(True)
plt.tight_layout()
plt.show()


#2. Intervalo entre lançamentos e nota média

#Não foi observada uma relação linear entre o intervalo de tempo entre lançamentos e a nota média #dos livros. Autores que publicam com intervalos curtos (entre um e três anos) apresentam desempenho #semelhante àqueles com intervalos mais longos. Isso sugere que o tempo entre publicações, por si #só, não é um fator determinante para avaliações melhores. A consistência na produção pode ter papel #mais relevante que pausas prolongadas.





# ===========================================
# 📊 GOLD - QTD DE LIVROS VS NOTA MÉDIA POR AUTOR (COM OUTLIERS E TENDÊNCIA)
# ===========================================
from pyspark.sql.functions import count, avg, round, col
import matplotlib.pyplot as plt
import numpy as np

# 1. Carregar tabela de livros por sequência
df_sequence = spark.table("book_gold.books_ratings_by_sequence")

# 2. Agrupar por autor para obter total de livros e média de nota
df_author_summary = df_sequence.groupBy("author").agg(
    count("title").alias("qtd_livros"),
    round(avg("avg_rating"), 2).alias("media_avaliacao")
).filter((col("qtd_livros") >= 2) & (col("qtd_livros") <= 100))

# 3. Salvar como tabela auxiliar
df_author_summary.write.mode("overwrite").saveAsTable("book_gold.books_volume_vs_avg")

# 4. Visualização: gráfico de dispersão com linha de tendência e destaque de outliers
# Converter para pandas
df_pd = df_author_summary.toPandas()

plt.figure(figsize=(10, 6))

# Dispersão
plt.scatter(df_pd["qtd_livros"], df_pd["media_avaliacao"], alpha=0.6, label="Autores")

# Linha de tendência (regressão linear)
z = np.polyfit(df_pd["qtd_livros"], df_pd["media_avaliacao"], 1)
p = np.poly1d(z)
plt.plot(df_pd["qtd_livros"], p(df_pd["qtd_livros"]), "r--", label="Tendência")

# Outliers: autores com 80+ livros e média > 4.5
outliers = df_pd[(df_pd["qtd_livros"] >= 80) & (df_pd["media_avaliacao"] > 4.5)]
plt.scatter(outliers["qtd_livros"], outliers["media_avaliacao"], color='orange', edgecolors='black', label="Outliers")

# ===========================================
# 📈 VISUALIZAÇÃO
# ===========================================
plt.title("Quantidade de Livros vs Média de Avaliação por Autor (até 100 livros)")
plt.xlabel("Quantidade de Livros")
plt.ylabel("Média das Notas")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

#3. Quantidade de livros publicados e nota média

#Há uma leve tendência de que autores com maior número de livros publicados apresentem uma média de #avaliação um pouco inferior. Ainda assim, alguns autores com alta produtividade conseguem manter #notas elevadas, aparecendo como exceções. Isso pode indicar que, embora a manutenção da qualidade #com volume alto seja desafiadora, ela é possível para certos perfis de autores.





# ===========================================
# 📊 GOLD - IDADE DO LEITOR VS ANO DO LIVRO LIDO
# ===========================================
from pyspark.sql.functions import col, floor, avg, count, when
import matplotlib.pyplot as plt

# 1. Carregar tabelas silver
books = spark.table("book_silver.books")
ratings = spark.table("book_silver.ratings")
users = spark.table("book_silver.users")

# 2. Join completo
ratings_books = ratings.join(books, on="isbn", how="inner")
rating_data = ratings_books.join(users, on="user_id", how="inner")

# 3. Filtrar dados consistentes
rating_clean = rating_data.filter(
    (col("age").isNotNull()) & (col("age") >= 10) & (col("age") <= 100) &
    (col("year").isNotNull()) & (col("year") >= 1450) & (col("year") <= 2025)
)

# 4. Criar faixa etária
rating_with_faixa = rating_clean.withColumn(
    "faixa_etaria", (floor(col("age") / 10) * 10).cast("int")
)

# 5. Agrupar por faixa etária e calcular média do ano do livro
df_idade_vs_ano = rating_with_faixa.groupBy("faixa_etaria").agg(
    round(avg("year"), 1).alias("media_ano_lido"),
    count("isbn").alias("total_livros_lidos")
).orderBy("faixa_etaria")

# 6. Salvar como tabela auxiliar
df_idade_vs_ano.write.mode("overwrite").saveAsTable("book_gold.reader_age_vs_book_year")

# ===========================================
# 📈 VISUALIZAÇÃO
# ===========================================
# Converter para pandas
df_pd = df_idade_vs_ano.toPandas()

plt.figure(figsize=(10, 5))
plt.plot(df_pd["faixa_etaria"], df_pd["media_ano_lido"], marker='o')
plt.title("Média do Ano dos Livros Lidos por Faixa Etária do Leitor")
plt.xlabel("Faixa Etária")
plt.ylabel("Ano Médio dos Livros Lidos")
plt.grid(True)
plt.tight_layout()
plt.show()

#4. Faixa etária do leitor e ano do livro lido

#Os dados mostram que leitores mais jovens tendem a ler livros mais recentes, enquanto leitores com #mais idade consomem obras publicadas há mais tempo. Essa relação entre faixa etária e época da obra #lida pode refletir preferências geracionais ou maior familiaridade com obras de diferentes períodos.





# ===========================================
# 📊 GOLD - AVALIAÇÃO POR IDADE DO LEITOR E ANO DO LIVRO (SUAVIZAÇÃO + ANOTAÇÕES)
# ===========================================
from pyspark.sql.functions import col, floor, avg, count, round, expr
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.ndimage import uniform_filter

# 1. Carregar tabelas base
books = spark.table("book_silver.books")
ratings = spark.table("book_silver.ratings")
users = spark.table("book_silver.users")

# 2. Join completo
ratings_books = ratings.join(books, on="isbn", how="inner")
rating_data = ratings_books.join(users, on="user_id", how="inner")

# 3. Filtrar dados válidos
rating_clean = rating_data.filter(
    (col("age").isNotNull()) & (col("age") >= 10) & (col("age") <= 100) &
    (col("year").isNotNull()) & (col("year") >= 1450) & (col("year") <= 2025)
)

# 4. Criar faixas
rating_with_bins = rating_clean.withColumn("faixa_etaria", (floor(col("age") / 10) * 10).cast("int")) \
    .withColumn("ano_livro", (floor(col("year") / 10) * 10).cast("int"))

# 5. Agrupar e calcular médias
df_avaliacoes = rating_with_bins.groupBy("faixa_etaria", "ano_livro").agg(
    round(avg("rating"), 2).alias("avg_rating"),
    count("rating").alias("total_avaliacoes")
).orderBy("faixa_etaria", "ano_livro")

# 6. Salvar tabela gold auxiliar
df_avaliacoes.write.mode("overwrite").saveAsTable("book_gold.age_book_year_rating")

# 7. Visualização: mapa de calor com suavização e anotações
# Converter para pandas
pdf = df_avaliacoes.toPandas()

# Pivot para heatmap
pivot = pdf.pivot(index="faixa_etaria", columns="ano_livro", values="avg_rating")

# Aplicar suavização com filtro de média (janela 3x3)
smoothed = uniform_filter(pivot.fillna(0), size=3)

# Criar DataFrame suavizado com mesmos índices para anotação
smoothed_df = pd.DataFrame(smoothed, index=pivot.index, columns=pivot.columns)

# ===========================================
# 📈 VISUALIZAÇÃO
# ===========================================
plt.figure(figsize=(14, 6))
sns.heatmap(smoothed_df, annot=smoothed_df.round(2), fmt=".2f", cmap="YlGnBu", linewidths=0.5)
plt.title("Média de Avaliação por Faixa Etária do Leitor e Ano do Livro (Suavizado)")
plt.xlabel("Ano do Livro (décadas)")
plt.ylabel("Faixa Etária do Leitor")
plt.tight_layout()
plt.show()

#6. Diversidade de autores por faixa etária

#Leitores com idade entre 30 e 45 anos tendem a ter contato com uma maior variedade de autores, #especialmente nas faixas entre 12 e 24 autores distintos. Por outro lado, leitores mais jovens (até #20 anos) e mais velhos (acima de 65) demonstram menor diversidade. Isso sugere que, na fase adulta, #há maior disposição para explorar diferentes estilos e autores.



# ===========================================
# 📊 GOLD - DISTRIBUIÇÃO DE DIVERSIDADE DE AUTORES POR FAIXA ETÁRIA DO LEITOR
# ===========================================
from pyspark.sql.functions import col, floor, countDistinct
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 1. Carregar tabelas silver
books = spark.table("book_silver.books")
ratings = spark.table("book_silver.ratings")
users = spark.table("book_silver.users")

# 2. Join completo
ratings_books = ratings.join(books, on="isbn", how="inner")
rating_data = ratings_books.join(users, on="user_id", how="inner")

# 3. Filtrar dados válidos
rating_clean = rating_data.filter(
    (col("age").isNotNull()) & (col("age") >= 10) & (col("age") <= 100) &
    (col("author").isNotNull()) & (col("author") != "")
)

# 4. Criar faixa etária de 5 em 5 anos
rating_with_bins = rating_clean.withColumn("faixa_etaria", (floor(col("age") / 5) * 5).cast("int"))

# 5. Calcular autores distintos por leitor
user_author_counts = rating_with_bins.groupBy("user_id", "faixa_etaria").agg(
    countDistinct("author").alias("autores_distintos")
)

# 6. Aplicar limite e criar faixas de diversidade de autores (3 em 3)
auth_bin_df = user_author_counts.filter(col("autores_distintos") <= 30) \
    .withColumn("faixa_autores", (floor(col("autores_distintos") / 3) * 3).cast("int"))

# 7. Contar usuários por faixa de idade e faixa de diversidade
distribuicao_df = auth_bin_df.groupBy("faixa_etaria", "faixa_autores").count().orderBy("faixa_etaria", "faixa_autores")

# 8. Converter para pandas e pivotar
pdf = distribuicao_df.toPandas()
pivot = pdf.pivot(index="faixa_autores", columns="faixa_etaria", values="count").fillna(0)

# ===========================================
# 📈 VISUALIZAÇÃO
# ===========================================
# 9. Visualizar como heatmap
plt.figure(figsize=(14, 6))
sns.heatmap(pivot, annot=True, fmt=".0f", cmap="YlOrBr", linewidths=0.5)
plt.title("Distribuição de Diversidade de Autores por Faixa Etária do Leitor (até 30 autores, agrupado de 3 em 3)")
plt.xlabel("Faixa Etária (anos)")
plt.ylabel("Número de Autores Distintos (agrupado de 3 em 3)")
plt.tight_layout()
plt.show()

#6. Diversidade de autores por faixa etária

#Leitores com idade entre 30 e 45 anos tendem a ter contato com uma maior variedade de autores, #especialmente nas faixas entre 12 e 24 autores distintos. Por outro lado, leitores mais jovens (até #20 anos) e mais velhos (acima de 65) demonstram menor diversidade. Isso sugere que, na fase adulta, #há maior disposição para explorar diferentes estilos e autores.




# ===========================================
# 📊 GOLD - CLUSTERIZAÇÃO DE AUTORES
# ===========================================
from pyspark.sql.functions import col, avg, stddev, count, round
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.clustering import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 1. Carregar tabela de sequência com notas por livro
df = spark.table("book_gold.books_ratings_by_sequence")

# 2. Agregar por autor para gerar métricas
autores_df = df.groupBy("author").agg(
    count("title").alias("qtd_livros"),
    round(avg("avg_rating"), 2).alias("media_nota"),
    round(stddev("avg_rating"), 2).alias("desvio_nota"),
    round(avg("year"), 0).alias("ano_medio")
).filter(col("qtd_livros") >= 3)

# 3. Vetorizar para ML
assembler = VectorAssembler(
    inputCols=["qtd_livros", "media_nota", "desvio_nota", "ano_medio"],
    outputCol="features"
)
vectorized_df = assembler.transform(autores_df)

# 4. Padronizar os dados
scaler = StandardScaler(inputCol="features", outputCol="scaled", withMean=True, withStd=True)
scaler_model = scaler.fit(vectorized_df)
scaled_df = scaler_model.transform(vectorized_df)

# 5. Aplicar KMeans (definindo 4 clusters)
kmeans = KMeans(featuresCol="scaled", predictionCol="cluster", k=4, seed=42)
model = kmeans.fit(scaled_df)
clustered = model.transform(scaled_df)

# 6. Converter para pandas para visualização
pdf = clustered.select("author", "qtd_livros", "media_nota", "desvio_nota", "ano_medio", "cluster").toPandas()

# ===========================================
# 📈 VISUALIZAÇÃO
# ===========================================
plt.figure(figsize=(10, 6))
sns.scatterplot(data=pdf, x="qtd_livros", y="media_nota", hue="cluster", palette="Set2")
plt.title("Clusterização de Autores por Volume e Nota Média")
plt.xlabel("Quantidade de Livros")
plt.ylabel("Média das Notas")
plt.tight_layout()
plt.show()

# 8. Salvar tabela com clusters
spark.createDataFrame(pdf).write.mode("overwrite").saveAsTable("book_gold.authors_clustered")

#7. Clusterização de autores

#A segmentação dos autores com base em volume de livros publicados, nota média, desvio das #avaliações e ano médio das obras permitiu identificar perfis distintos. Foram encontrados #agrupamentos com autores prolíficos e bem avaliados, autores com alta variabilidade nas avaliações #e autores mais recentes com desempenho positivo. Esta análise permite mapear padrões de atuação no #cenário literário.




# ===========================================
# 🌍 GOLD - CLUSTERIZAÇÃO DE PAÍSES POR FAIXAS ETÁRIAS (PROPORÇÃO POR FAIXA ORDENADO POR TOTAL)
# ===========================================
from pyspark.sql.functions import col, lower, split, trim, regexp_replace, count, floor, sum as sum_
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 1. Carregar tabela de usuários
users = spark.table("book_silver.users")

# 2. Extrair país após a segunda vírgula
users_country = users.withColumn("country_raw", split(col("location"), ",").getItem(2)) \
    .withColumn("country", regexp_replace(trim(lower(col("country_raw"))), "[^a-z ]", "")) \
    .filter(col("country").isNotNull() & (col("country") != ""))

# 3. Filtrar idades válidas
users_valid_age = users_country.filter((col("age") >= 10) & (col("age") <= 100))

# 4. Criar faixa etária de 5 em 5 anos
users_with_bins = users_valid_age.withColumn("faixa_etaria", (floor(col("age") / 5) * 5).cast("int"))

# 5. Contar número de leitores por país e faixa etária
df_country_faixa = users_with_bins.groupBy("country", "faixa_etaria").agg(count("user_id").alias("total"))

# 6. Selecionar os 30 países com mais leitores no total
total_by_country = df_country_faixa.groupBy("country").agg(sum_("total").alias("total_pais"))
top30_countries = total_by_country.orderBy(col("total_pais").desc()).limit(30)
df_top30 = df_country_faixa.join(top30_countries.select("country"), on="country", how="inner")

# 7. Calcular proporção por faixa etária em cada país
df_top30_total = df_top30.join(total_by_country.withColumnRenamed("total_pais", "total_pais_all"), on="country")
df_top30_prop = df_top30_total.withColumn("proporcao", (col("total") / col("total_pais_all")).cast("double"))

# 8. Pivotar proporções por faixa etária
df_pivot = df_top30_prop.groupBy("country").pivot("faixa_etaria").agg({"proporcao": "avg"}).na.fill(0)

# 9. Clusterização com KMeans por distribuição etária proporcional
features = [col for col in df_pivot.columns if col != "country"]
assembler = VectorAssembler(inputCols=features, outputCol="features")
data_vector = assembler.transform(df_pivot)

kmeans = KMeans(k=4, seed=42, featuresCol="features", predictionCol="cluster")
model = kmeans.fit(data_vector)
clustered = model.transform(data_vector)

# ===========================================
# 📈 VISUALIZAÇÃO
# ===========================================
# 10. Expandir para gráfico de bolhas com proporção e ordenação por total
df_expanded = df_top30_prop.join(clustered.select("country", "cluster"), on="country", how="inner")
df_expanded = df_expanded.join(top30_countries, on="country")
pdf_bubbles = df_expanded.select("country", "faixa_etaria", "proporcao", "cluster", "total_pais").toPandas()
pdf_bubbles["country"] = pd.Categorical(pdf_bubbles["country"], 
    categories=pdf_bubbles.groupby("country")["total_pais"].max().sort_values(ascending=False).index,
    ordered=True
)

# 11. Visualização com gráfico de bolhas
plt.figure(figsize=(14, 6))
sns.scatterplot(
    data=pdf_bubbles,
    x="faixa_etaria",
    y="country",
    size="proporcao",
    hue="cluster",
    sizes=(40, 800),
    alpha=0.7,
    palette="Set2"
)
plt.title("Distribuição Proporcional de Leitores por Faixa Etária e País (Ordenado por Volume)")
plt.xlabel("Faixa Etária")
plt.ylabel("País (ordenado por total de leitores)")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()
plt.show()

#8. Clusterização de países por faixa etária

#A análise por país, considerando a proporção de leitores por faixa etária, mostrou perfis distintos #entre as nações. Países como Estados Unidos, Índia e Alemanha apresentaram maior equilíbrio etário, #enquanto outros se destacaram por concentrações em faixas mais jovens. Essa informação pode indicar #variações no hábito de leitura e ajudar a identificar oportunidades de atuação segmentada por #região.
